# W261 Final Project ETL for Development Sample

### Notebook Set-Up

In [1]:
# imports
import time
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import DataFrame

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]
PWD

'/media/notebooks'

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261FinalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

## Load Data

In [5]:
original_trainRDD = sc.textFile('data/train.txt')
original_testRDD = sc.textFile('data/test.txt')

## Transformation

### Sample

In [6]:
#change the seed for a different sample
sampleRDD1, sampleRDD2 = original_trainRDD.randomSplit([0.9999,0.0001], seed = 1)
sampleRDD2.cache()

PythonRDD[4] at RDD at PythonRDD.scala:49

In [7]:
ncol = len(sampleRDD2.take(1)[0].split('\t'))
nrow = sampleRDD2.count()
print("This sample contains", str(nrow), "rows.")

This sample contains 4478 rows.


In [8]:
print("This sample contains", str(ncol), "columns.")

This sample contains 40 columns.


In [9]:
def avgFeatures(line):
    
    count = 0
    feats = line.split('\t')[1:]
    
    for feat in feats:
        if feat != '':
            count += 1

    return count

print("There is an average of", str(round(sampleRDD2.map(avgFeatures).mean(),2)), "populated features per observation.")
sampleRDD2.map(avgFeatures).mean()

There is an average of 33.53 populated features per observation.


33.5288075033497

# Put in wide, sparse feature format

In [10]:
def parseCV(line):
    """
    Map record_csv_string --> (features, label)
    """

    # start of categorical features
    col_start = 14
    
    raw_values = line.split('\t')
    label = int(raw_values[0])
    
    # ignore numerics to start
    #numerical_values = list(pd.Series(raw_values[1:14]).apply(pd.to_numeric))
    numericals = []
    for idx, value in enumerate(raw_values[1:col_start]):
        if value != '':
            numericals.append('n' + str(idx) + '_' + str(value))
            
    
    categories = []
    for idx, value in enumerate(raw_values[col_start:]):
        if value != '':
            categories.append('c'+ str(idx) + '_' + str(value))

    return Row(label=label, raw=numericals + categories)


def vectorizeCV(DF):
    
    vectorizer = CountVectorizer()
    cv = CountVectorizer(inputCol="raw", outputCol="features", binary=True)
    
    model = cv.fit(DF)
    result = model.transform(DF)
    num_feats = cv.getVocabSize()
    
    return result


parsedDF = sampleRDD2.map(parseCV).toDF().cache()
vectorizedDF = vectorizeCV(parsedDF)

In [11]:
vectorizedDF.show(truncate=True)

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(30946,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(30946,[0,1,2,5,1...|
|    1|[n1_1, n2_1, n4_9...|(30946,[0,1,6,7,1...|
|    0|[n0_8, n1_17, n3_...|(30946,[0,1,4,12,...|
|    1|[n0_6, n1_1, n2_7...|(30946,[0,1,2,4,1...|
|    1|[n1_99, n2_1, n3_...|(30946,[1,2,4,10,...|
|    0|[n0_3, n1_21, n2_...|(30946,[0,1,4,8,1...|
|    0|[n1_2, n2_20, n3_...|(30946,[0,1,3,5,8...|
|    0|[n0_0, n1_144, n4...|(30946,[0,2,3,4,5...|
|    0|[n1_0, n2_5, n4_3...|(30946,[0,2,3,6,1...|
|    0|[n0_0, n1_1, n2_4...|(30946,[0,1,2,3,5...|
|    0|[n0_9, n1_5, n2_1...|(30946,[0,2,3,6,9...|
|    0|[n1_323, n2_2, n3...|(30946,[1,2,14,16...|
|    0|[n0_0, n1_424, n3...|(30946,[0,1,2,4,6...|
|    0|[n0_0, n1_13, n2_...|(30946,[0,1,2,5,6...|
|    0|[n1_180, n2_6, n3...|(30946,[1,2,8,14,...|
|    0|[n1_126, n2_2, n3...|(30946,[0,2,4,6,8...|


In [12]:
vectorizedRDD = vectorizedDF.select(['label', 'features']).rdd.cache()
vectorizedRDD.take(1)

[Row(label=0, features=SparseVector(30946, {0: 1.0, 1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 20: 1.0, 32: 1.0, 122: 1.0, 154: 1.0, 173: 1.0, 214: 1.0, 365: 1.0, 370: 1.0, 494: 1.0, 504: 1.0, 628: 1.0, 634: 1.0, 833: 1.0, 1943: 1.0, 2065: 1.0, 2076: 1.0, 2233: 1.0, 2774: 1.0, 6172: 1.0, 11334: 1.0, 14351: 1.0, 18114: 1.0, 18353: 1.0, 19838: 1.0, 24028: 1.0, 25370: 1.0, 26552: 1.0}))]

In [13]:
# feature data struct tester
num_feats = vectorizedRDD.take(1)[0][1].size
num_feats

30946

In [14]:
vectorizedRDD.map(lambda x: x[0]).mean()

0.25949084412684253

## test prediction making

In [21]:
#sc.addPyFile("./fm_function_tester.py")
#import fm_function_tester as fm
#from fm_function_tester import *

In [67]:
# initialize weights
np.random.seed(24)

b_br = sc.broadcast(0.0)
w_br = sc.broadcast(np.zeros(num_feats, dtype=np.float16))
k_br = sc.broadcast(2)
V_br = sc.broadcast(np.random.normal(0.0, 0.5, (k_br.value, num_feats)))
V_br.value.shape

(2, 30946)

In [17]:
import numpy as np
from pyspark.ml.feature import CountVectorizer
from scipy.sparse import csr_matrix

def predict_grad(pair):
    """
        Compute the predicted probability AND return the gradient (?)
        Args:
            pair - records are in (label, sparse feature set) format
        Broadcast:
            b - bias term (scalar)
            w - linear weight vector (array)
            k - number of factors (def=2)
            V - factor matrix of size (d dimensions, k=2 factors)
        Returns:
            predRDD - pair of ([label, predicted probability], feature set)
    """
    
    label = pair[0]
    feats = pair[1]
    
    # start with linear weight dot product
    linear_sum = np.dot(w_br.value[feats.indices], feats.values)
    
    #linear_sum = 0.0
    #for i in feats.indices:
    #    i = int(i)
    #    linear_sum += w_br.value[i]*feats[i]

    # factor matrix interaction sum
    factor_sum = 0.0
    lh_factor = [0.0]*k_br.value
    rh_factor = [0.0]*k_br.value
    
    for f in range(0, k_br.value):
        
        lh_factor[f] = np.dot(V_br.value[f][feats.indices], feats.values)  #KEY--this is used in v_grad matrix below
        rh_factor[f] = np.dot(V_br.value[f][feats.indices]**2, feats.values**2)
        
        #for i in feats.indices: #old with extra for loop
        #    i = int(i)
        #    lh_factor[f] += V_br.value[i][f]*feats[i]  #KEY--this is used in v_grad matrix below
        #    rh_factor += (V_br.value[i][f]**2) * (feats[i]**2)
        
        factor_sum += (lh_factor[f]**2 - rh_factor[f])
    factor_sum = 0.5 * factor_sum
    
    preProb = b_br.value + linear_sum + factor_sum
    
    prob = 1.0 / (1 + np.exp(-preProb))  #logit transformation
    
    #compute Gradients
    b_grad = label - prob
    
    w_grad = csr_matrix((b_grad*feats.values, feats.indices, np.array([0, feats.indices.size])), (1, w_br.value.shape[0]))
    #print(w_grad[(0,5)])
    
    v_grad = csr_matrix((V_br.value.shape[0], V_br.value.shape[1]))
    for f in range(0, k_br.value):  # WORKING
        for i in feats.indices:
            i = int(i)
            v_grad[(f,i)] = b_grad * (feats[i]*lh_factor[f] - V_br.value[f][i]*(feats[i]**2))
    #print(type(v_grad))
    
    
    return ([label, prob], [b_grad, w_grad, v_grad])

In [53]:
# toy example
tester_grads = predict_grad(vectorizedRDD.take(1)[0])
sparse_mtx = tester_grads[1][2]
sparse_mtx[(0,5)]

/opt/anaconda/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


0.004292632661100083

In [74]:
predRDD = vectorizedRDD.map(predict_grad).cache()
test_pred = predRDD.take(1)
test_pred

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 147.0 failed 1 times, most recent failure: Lost task 0.0 in stage 147.0 (TID 29101, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-52-b1c5b250582b>", line 23, in predict_grad
  File "/opt/anaconda/lib/python3.6/site-packages/numpy/matrixlib/d

In [73]:
b_br.value

0.0011070609323269635

### Get log-loss on the sample set

In [18]:
from sklearn.metrics import log_loss

In [56]:
from sklearn.metrics import log_loss

sample_preds = np.array(predRDD.map(lambda x: x[0]).take(20))
sample_loss = log_loss(sample_preds[:,0], sample_preds[:,1], normalize=True)
sample_loss

2.6458644290629403

## Reduce sparse vectors, get mean of gradients, and update parameter vectors

In [ ]:
############## initialize weights ##############
# initialize 
b = 0.0
w = csr_matrix(np.zeros(num_feats, dtype=np.float16))
k = 2
V = csr_matrix(np.random.normal(0.0, 0.5, (k_br.value, num_feats)))

# broadcast
b_br = sc.broadcast(b)
w_br = sc.broadcast(w)
k_br = sc.broadcast(k)
V_br = sc.broadcast(V)

############## gradient calculation ##############
# calculate average gradient for b
bGrad = predRDD.map(lambda x: x[1][0]).mean()

# calculate average gradient for w
N = predRDD.count()
wGrad = (1/N) * predRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b)

# calculate average gradient for V
vGrad = (1/N) * predRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b)

############## update weights ##############
# update
b -= 0.01 * bGrad
w -= 0.01 * wGrad
V -= 0.01 * vGrad

# re-broadcast
b_br = sc.broadcast(b)
w_br = sc.broadcast(w)
V_br = sc.broadcast(V)


In [69]:
############## gradient calculation ##############
# calculate average gradient for b
bGrad = predRDD.map(lambda x: x[1][0]).mean()

# calculate average gradient for w
N = predRDD.count()
wGrad = (1/N) * predRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b)

# calculate average gradient for V
vGrad = (1/N) * predRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b)

############## update weights ##############
# update
b -= 0.01 * bGrad
w -= 0.01 * wGrad
V -= 0.01 * vGrad

# re-broadcast
b_br = sc.broadcast(b)
w_br = sc.broadcast(w)
V_br = sc.broadcast(V)

In [70]:
b

0.0011070609323269635

In [71]:
b_br.value

0.0011070609323269635

In [78]:
w

matrix([[8.30160213e-04, 7.17106039e-04, 6.34518692e-04, ...,
         2.11623696e-08, 2.12899741e-06, 3.20285805e-10]])

## Iterate predictions and parameter updates

In [30]:
# initialize weights
np.random.seed(24)

b = 0.0
w = np.zeros(num_feats, dtype=np.float16)
k = 2
#V = np.random.normal(0.0, 0.5, (k_br.value, num_feats))
V = np.random.normal(0.0, 0.5, (k, num_feats))

b_br = sc.broadcast(b)
w_br = sc.broadcast(w)
k_br = sc.broadcast(k)
V_br = sc.broadcast(V)
V_br.value.shape

(2, 30946)

In [30]:
## initialize
#b = 0.0
#w = csr_matrix(np.zeros(num_feats, dtype=np.float16))
#k = 2
#V = csr_matrix(np.random.normal(0.0, 0.5, (k_br.value, num_feats)))

# broadcast
#b_br = sc.broadcast(b)
#w_br = sc.broadcast(w)
#k_br = sc.broadcast(k)
#V_br = sc.broadcast(V)


In [41]:
def train(dataRDD, numIterations=5, learningRate = 0.01):
    
    def predict_grad(pair):
        """
            Compute the predicted probability AND return the gradient (?)
            Args:
                pair - records are in (label, sparse feature set) format
            Broadcast:
                b - bias term (scalar)
                w - linear weight vector (array)
                k - number of factors (def=2)
                V - factor matrix of size (d dimensions, k=2 factors)
            Returns:
                predRDD - pair of ([label, predicted probability], feature set)
        """

        label = pair[0]
        feats = pair[1]

        # start with linear weight dot product
        linear_sum = np.dot(w_br.value[feats.indices], feats.values)

        #linear_sum = 0.0
        #for i in feats.indices:
        #    i = int(i)
        #    linear_sum += w_br.value[i]*feats[i]

        # factor matrix interaction sum
        factor_sum = 0.0
        lh_factor = [0.0]*k_br.value
        rh_factor = [0.0]*k_br.value

        for f in range(0, k_br.value):

            lh_factor[f] = np.dot(V_br.value[f][feats.indices], feats.values)  #KEY--this is used in v_grad matrix below
            rh_factor[f] = np.dot(V_br.value[f][feats.indices]**2, feats.values**2)

            #for i in feats.indices: #old with extra for loop
            #    i = int(i)
            #    lh_factor[f] += V_br.value[i][f]*feats[i]  #KEY--this is used in v_grad matrix below
            #    rh_factor += (V_br.value[i][f]**2) * (feats[i]**2)

            factor_sum += (lh_factor[f]**2 - rh_factor[f])
        factor_sum = 0.5 * factor_sum

        preProb = b_br.value + linear_sum + factor_sum

        prob = 1.0 / (1 + np.exp(-preProb))  #logit transformation

        #compute Gradients
        b_grad = label - prob

        w_grad = csr_matrix((b_grad*feats.values, feats.indices, np.array([0, feats.indices.size])), (1, w_br.value.shape[0]))
        #print(w_grad[(0,5)])

        v_grad = csr_matrix((V_br.value.shape[0], V_br.value.shape[1]))
        for f in range(0, k_br.value):  # WORKING
            for i in feats.indices:
                i = int(i)
                v_grad[(f,i)] = b_grad * (feats[i]*lh_factor[f] - V_br.value[f][i]*(feats[i]**2))
        #print(type(v_grad))


        return ([label, prob], [b_grad, w_grad, v_grad])
    # initialize weights
    np.random.seed(24)

    b = 0.0
    w = np.zeros(num_feats, dtype=np.float16)
    k = 2
    V = np.random.normal(0.0, 0.5, (k, num_feats))

    b_br = sc.broadcast(b)
    w_br = sc.broadcast(w)
    k_br = sc.broadcast(k)
    V_br = sc.broadcast(V)
    #b_br.value
    #w_br.value.shape
    #k_br.value
    #V_br.value.shape
    
    for n in range(numIterations):
        
        predictionRDD = dataRDD.map(predict_grad).cache()
        #([label, prob], [b_grad, w_grad, v_grad])
        

        ############## gradient calculation ##############
        # calculate average gradient for b
        bGrad = predictionRDD.map(lambda x: x[1][0]).mean()

        # calculate average gradient for w
        N = predictionRDD.count()
        #wGrad = (1/N) * predictionRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b)
        wGrad = np.multiply((1/N),predictionRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b))

        # calculate average gradient for V
        #vGrad = (1/N) * predictionRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b)
        vGrad = np.multiply((1/N),predictionRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b))

        
        b_br.unpersist()
        w_br.unpersist()
        V_br.unpersist()
        ############## update weights ##############
        ## update
        #b -= learningRate * bGrad
        #w -= learningRate * wGrad
        #V -= learningRate * vGrad
        
        b = np.subtract(b, np.multiply(learningRate, bGrad))
        w = np.subtract(w, np.multiply(learningRate, wGrad))
        V = np.subtract(V, np.multiply(learningRate, vGrad))
        
        print("b:", str(b))

        # re-broadcast
        b_br = sc.broadcast(b)
        w_br = sc.broadcast(w)
        V_br = sc.broadcast(V)
        
        #track loss
        sample_preds = np.array(predictionRDD.map(lambda x: x[0]).take(20))
        sample_loss = log_loss(sample_preds[:,0], sample_preds[:,1], normalize=True)
        #print("sample loss:", str(np.exp(sample_loss)))
        print("sample loss:", str(sample_loss))
        predictionRDD.unpersist()
        
    return predictionRDD

In [42]:
predRDD = train(vectorizedRDD, numIterations=2, learningRate=0.01)

SystemError: <built-in function isinstance> returned a result with an error set

In [45]:
from sklearn.metrics import log_loss

sample_preds = np.array(predRDD.map(lambda x: x[0]).take(20))
sample_loss = log_loss(sample_preds[:,0], sample_preds[:,1], normalize=True)
sample_loss

2.6458644290629403

In [39]:
b

0.0

In [40]:
b_br.value

0.0

In [88]:
w_br.value

array([0., 0., 0., ..., 0., 0., 0.], dtype=float16)